In [18]:
# Dependencies
import pandas as pd
import hvplot.pandas
import requests
import matplotlib.pyplot as plt
import numpy as np
import json

# Import API key
from api_keys import geoapify_key

In [19]:
# The correct encoding must be used to read the CSV in pandas
kyoto = pd.read_csv("Resources/Kyoto_Restaurant_Info.csv")

# Display sample data
kyoto_df=pd.DataFrame(kyoto)
kyoto_df


,Unnamed: 0,Name,JapaneseName,Station,FirstCategory,SecondCategory,DinnerPrice,LunchPrice,TotalRating,DinnerRating,LunchRating,ReviewNum,Lat,Long
0,1,Orudeidainingurajou,オールデイダイニング ラジョウ,Kyoto,Buffet style,Cafe,￥4000～￥4999,￥2000～￥2999,3.39,3.20,3.38,56,34.986175,135.761314
1,2,Steak Frites Gaspard zinzin,ステックフリット ガスパール ザンザン,Karasuma,Bistro,Steak,￥3000～￥3999,￥1000～￥1999,3.18,3.06,3.33,70,35.005826,135.759857
2,3,KAZUMA,和馬,Sanjo,Izakaya (Tavern),Japanese food (other),￥3000～￥3999,NaN,3.28,3.28,NaN,7,35.006583,135.770247
3,4,okonomiyakiteppanyakimiki,お好み焼き 鉄板焼き 三喜,Tambaguchi,Okonomiyaki,Izakaya (Tavern),￥3000～￥3999,NaN,3.14,3.14,NaN,16,34.995924,135.748267
4,5,Shaofeiyan,小肥羊 京都河原町店,Kyoto Shiyakusho Mae,Chinese hot pot / fire pot,Chinese,￥4000～￥4999,￥1000～￥1999,3.16,3.16,3.05,23,35.007901,135.768202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,891,Waraku,和楽,Fukuchiyama,Izakaya (Tavern),Kaiseki (Traditional Japanese),￥2000～￥2999,～￥999,3.00,3.06,3.00,12,35.298917,135.118341
891,892,YEBISU BAR,エビスバー 京都ヨドバシ店,Kyoto,Beer bar,Bar,￥3000～￥3999,～￥999,3.08,3.07,3.08,70,34.988132,135.759215
892,893,Ibushiginkazuya,燻吟 かず家,Kawaramachi,Bar,Nihonshu (Japanese sake),￥3000～￥3999,NaN,3.36,3.36,3.05,11,35.006449,135.769653
893,894,Tenichi,天一 京都伊勢丹店,Kyoto,Tempura,Ten-don (Tempura Bowl),￥6000～￥7999,￥2000～￥2999,3.51,3.32,3.53,52,34.985887,135.757672


# Clean up data frame

In [20]:
# Delete extraneous column
kyoto_df = kyoto_df.drop(columns = ["JapaneseName", "DinnerRating", "LunchRating"])
kyoto_df.head()

,Unnamed: 0,Name,Station,FirstCategory,SecondCategory,DinnerPrice,LunchPrice,TotalRating,ReviewNum,Lat,Long
0,1,Orudeidainingurajou,Kyoto,Buffet style,Cafe,￥4000～￥4999,￥2000～￥2999,3.39,56,34.986175,135.761314
1,2,Steak Frites Gaspard zinzin,Karasuma,Bistro,Steak,￥3000～￥3999,￥1000～￥1999,3.18,70,35.005826,135.759857
2,3,KAZUMA,Sanjo,Izakaya (Tavern),Japanese food (other),￥3000～￥3999,NaN,3.28,7,35.006583,135.770247
3,4,okonomiyakiteppanyakimiki,Tambaguchi,Okonomiyaki,Izakaya (Tavern),￥3000～￥3999,NaN,3.14,16,34.995924,135.748267
4,5,Shaofeiyan,Kyoto Shiyakusho Mae,Chinese hot pot / fire pot,Chinese,￥4000～￥4999,￥1000～￥1999,3.16,23,35.007901,135.768202


# Combine First and second Category


In [21]:
kyoto_df['Category'] = kyoto_df[['FirstCategory', 'SecondCategory']].apply(lambda x: ' '.join(x.astype(str)), axis=1)

# Drop the original columns
kyoto_mergeCat_df = kyoto_df.drop(['FirstCategory', 'SecondCategory'], axis=1)

kyoto_mergeCat_df

,Unnamed: 0,Name,Station,DinnerPrice,LunchPrice,TotalRating,ReviewNum,Lat,Long,Category
0,1,Orudeidainingurajou,Kyoto,￥4000～￥4999,￥2000～￥2999,3.39,56,34.986175,135.761314,Buffet style Cafe
1,2,Steak Frites Gaspard zinzin,Karasuma,￥3000～￥3999,￥1000～￥1999,3.18,70,35.005826,135.759857,Bistro Steak
2,3,KAZUMA,Sanjo,￥3000～￥3999,NaN,3.28,7,35.006583,135.770247,Izakaya (Tavern) Japanese food (other)
3,4,okonomiyakiteppanyakimiki,Tambaguchi,￥3000～￥3999,NaN,3.14,16,34.995924,135.748267,Okonomiyaki Izakaya (Tavern)
4,5,Shaofeiyan,Kyoto Shiyakusho Mae,￥4000～￥4999,￥1000～￥1999,3.16,23,35.007901,135.768202,Chinese hot pot / fire pot Chinese
...,...,...,...,...,...,...,...,...,...,...
890,891,Waraku,Fukuchiyama,￥2000～￥2999,～￥999,3.00,12,35.298917,135.118341,Izakaya (Tavern) Kaiseki (Traditional Japanese)
891,892,YEBISU BAR,Kyoto,￥3000～￥3999,～￥999,3.08,70,34.988132,135.759215,Beer bar Bar
892,893,Ibushiginkazuya,Kawaramachi,￥3000～￥3999,NaN,3.36,11,35.006449,135.769653,Bar Nihonshu (Japanese sake)
893,894,Tenichi,Kyoto,￥6000～￥7999,￥2000～￥2999,3.51,52,34.985887,135.757672,Tempura Ten-don (Tempura Bowl)


# Soft ReviewNum Large to low

In [23]:
kyoto_softReviewNum_df = kyoto_mergeCat_df.sort_values("ReviewNum", ascending=False)
kyoto_softReviewNum_df

,Unnamed: 0,Name,Station,DinnerPrice,LunchPrice,TotalRating,ReviewNum,Lat,Long,Category
786,787,Honkedaiichiasahi,Kyoto,～￥999,～￥999,3.60,1319,34.986695,135.762508,Ramen Dumplings
660,661,chuukasobatakayasu,Ichijoji,～￥999,～￥999,3.59,713,35.045052,135.785180,Ramen Boneless deep-fried chicken
770,771,Itoukyuuemon,Uji,～￥999,NaN,3.58,380,34.896549,135.808757,Sweets Japan tea shoppe
38,39,Zasoudouhigashiyamakyouto,Gion Shijo,￥6000～￥7999,￥2000～￥2999,3.61,272,34.998896,135.779668,Italian Pasta
67,68,Kyoutogiontempurayasakaendou,Gion Shijo,￥10000～￥14999,￥5000～￥5999,3.68,252,34.998915,135.775112,Tempura Kaiseki (Traditional Japanese)
...,...,...,...,...,...,...,...,...,...,...
197,198,Ilifune,Jingu Marutamachi,￥3000～￥3999,NaN,3.06,1,35.016789,135.768654,Dining bar Italian
44,45,Shuteimichiya,Karasuma Oike,￥5000～￥5999,～￥999,3.06,1,35.013322,135.758725,Izakaya (Tavern) Seafood
634,635,Yakinikuhorumonfukukawa,Omiya,￥3000～￥3999,NaN,3.04,1,35.003922,135.749414,Yakiniku (BBQ Beef) nan
134,135,teppandainingugionshou,Gion Shijo,￥2000～￥2999,NaN,3.02,1,35.004029,135.772910,Izakaya (Tavern) Teppanyaki


# Soft TotalRating Large to low

In [24]:
kyoto_softReviewNum_df = kyoto_mergeCat_df.sort_values("TotalRating", ascending=False)
kyoto_softReviewNum_df

,Unnamed: 0,Name,Station,DinnerPrice,LunchPrice,TotalRating,ReviewNum,Lat,Long,Category
787,788,Kichisen,Demachiyanagi,￥20000～￥29999,￥10000～￥14999,4.19,79,35.035797,135.771352,Kaiseki (Traditional Japanese) nan
837,838,ORTO,Karasuma Oike,￥8000～￥9999,￥4000～￥4999,4.06,53,35.007991,135.757053,Innovative cuisine Fusion cuisine
123,124,Yosuke YAMAJI,Gion Shijo,￥20000～￥29999,￥8000～￥9999,4.02,44,35.002708,135.775670,French Modern French
384,385,Iharada,Tambaguchi,￥10000～￥14999,NaN,3.91,52,34.990530,135.732212,Kyoto Cuisine Kaiseki (Traditional Japanese)
657,658,nihonryourisakuragawa,Kyoto Shiyakusho Mae,￥20000～￥29999,￥8000～￥9999,3.89,61,35.012176,135.770684,Kaiseki (Traditional Japanese) Kappo (Traditio...
...,...,...,...,...,...,...,...,...,...,...
356,357,kyoumachiyaitarianronkusu,Kawaramachi,￥5000～￥5999,￥2000～￥2999,3.00,15,35.000321,135.766175,Italian Sumibiyaki
767,768,Washokumai,Kyoto,￥3000～￥3999,￥1000～￥1999,3.00,9,34.983869,135.761333,Tempura Sushi
610,611,Kinnotori,Kujo,￥3000～￥3999,NaN,3.00,4,34.981663,135.761245,Izakaya (Tavern) Yakitori (Grilled chicken)
113,114,Shisui,Sanjo,￥3000～￥3999,NaN,3.00,8,35.009196,135.770522,Izakaya (Tavern) Kyoto Cuisine


# Count Lunch Price in how many resturant

In [17]:
kyoto_df.count()

Unnamed: 0     895
Name           895
Station        895
DinnerPrice    895
LunchPrice     484
TotalRating    895
ReviewNum      895
Lat            895
Long           895
Category       895
dtype: int64

# Overall Summary_Statistics

In [15]:
kyoto_df.describe()

,Unnamed: 0,TotalRating,ReviewNum,Lat,Long
count,895.000000,895.000000,895.000000,895.000000,895.000000
mean,448.000000,3.212480,27.080447,35.005287,135.755774
std,258.508543,0.215588,58.064708,0.062367,0.056629
min,1.000000,3.000000,1.000000,34.715818,135.094284
25%,224.500000,3.050000,7.000000,34.995639,135.757501
50%,448.000000,3.090000,15.000000,35.004454,135.762578
75%,671.500000,3.420000,30.000000,35.008488,135.770655
max,895.000000,4.190000,1319.000000,35.735303,135.831994


# Dinner Price split up

In [28]:
DinnerPrice = kyoto_mergeCat_df["DinnerPrice"][0].split("～")
DinnerPrice[0]
kyoto_mergeCat_df["DinnerPriceLow"] = DinnerPrice[0]
kyoto_mergeCat_df["DinnerPriceHigh"] = DinnerPrice[1]
kyoto_mergeCat_df

,Unnamed: 0,Name,Station,DinnerPrice,LunchPrice,TotalRating,ReviewNum,Lat,Long,Category,DinnerPriceLow,DinnerPriceHigh
0,1,Orudeidainingurajou,Kyoto,￥4000～￥4999,￥2000～￥2999,3.39,56,34.986175,135.761314,Buffet style Cafe,￥4000,￥4999
1,2,Steak Frites Gaspard zinzin,Karasuma,￥3000～￥3999,￥1000～￥1999,3.18,70,35.005826,135.759857,Bistro Steak,￥4000,￥4999
2,3,KAZUMA,Sanjo,￥3000～￥3999,NaN,3.28,7,35.006583,135.770247,Izakaya (Tavern) Japanese food (other),￥4000,￥4999
3,4,okonomiyakiteppanyakimiki,Tambaguchi,￥3000～￥3999,NaN,3.14,16,34.995924,135.748267,Okonomiyaki Izakaya (Tavern),￥4000,￥4999
4,5,Shaofeiyan,Kyoto Shiyakusho Mae,￥4000～￥4999,￥1000～￥1999,3.16,23,35.007901,135.768202,Chinese hot pot / fire pot Chinese,￥4000,￥4999
...,...,...,...,...,...,...,...,...,...,...,...,...
890,891,Waraku,Fukuchiyama,￥2000～￥2999,～￥999,3.00,12,35.298917,135.118341,Izakaya (Tavern) Kaiseki (Traditional Japanese),￥4000,￥4999
891,892,YEBISU BAR,Kyoto,￥3000～￥3999,～￥999,3.08,70,34.988132,135.759215,Beer bar Bar,￥4000,￥4999
892,893,Ibushiginkazuya,Kawaramachi,￥3000～￥3999,NaN,3.36,11,35.006449,135.769653,Bar Nihonshu (Japanese sake),￥4000,￥4999
893,894,Tenichi,Kyoto,￥6000～￥7999,￥2000～￥2999,3.51,52,34.985887,135.757672,Tempura Ten-don (Tempura Bowl),￥4000,￥4999


# Loop the dinner price Low and high

In [30]:
for i in range(len(kyoto_mergeCat_df["DinnerPrice"])):
    DinnerPrice = kyoto_mergeCat_df["DinnerPrice"][i].split("～")
    kyoto_mergeCat_df["DinnerPriceLow"] = DinnerPrice[0]
    kyoto_mergeCat_df["DinnerPriceHigh"] = DinnerPrice[1]
    

In [31]:
kyoto_mergeCat_df

,Unnamed: 0,Name,Station,DinnerPrice,LunchPrice,TotalRating,ReviewNum,Lat,Long,Category,DinnerPriceLow,DinnerPriceHigh
0,1,Orudeidainingurajou,Kyoto,￥4000～￥4999,￥2000～￥2999,3.39,56,34.986175,135.761314,Buffet style Cafe,￥8000,￥9999
1,2,Steak Frites Gaspard zinzin,Karasuma,￥3000～￥3999,￥1000～￥1999,3.18,70,35.005826,135.759857,Bistro Steak,￥8000,￥9999
2,3,KAZUMA,Sanjo,￥3000～￥3999,NaN,3.28,7,35.006583,135.770247,Izakaya (Tavern) Japanese food (other),￥8000,￥9999
3,4,okonomiyakiteppanyakimiki,Tambaguchi,￥3000～￥3999,NaN,3.14,16,34.995924,135.748267,Okonomiyaki Izakaya (Tavern),￥8000,￥9999
4,5,Shaofeiyan,Kyoto Shiyakusho Mae,￥4000～￥4999,￥1000～￥1999,3.16,23,35.007901,135.768202,Chinese hot pot / fire pot Chinese,￥8000,￥9999
...,...,...,...,...,...,...,...,...,...,...,...,...
890,891,Waraku,Fukuchiyama,￥2000～￥2999,～￥999,3.00,12,35.298917,135.118341,Izakaya (Tavern) Kaiseki (Traditional Japanese),￥8000,￥9999
891,892,YEBISU BAR,Kyoto,￥3000～￥3999,～￥999,3.08,70,34.988132,135.759215,Beer bar Bar,￥8000,￥9999
892,893,Ibushiginkazuya,Kawaramachi,￥3000～￥3999,NaN,3.36,11,35.006449,135.769653,Bar Nihonshu (Japanese sake),￥8000,￥9999
893,894,Tenichi,Kyoto,￥6000～￥7999,￥2000～￥2999,3.51,52,34.985887,135.757672,Tempura Ten-don (Tempura Bowl),￥8000,￥9999
